In [1]:
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'retina'
import sys
sys.path.append('.venv/lib/python3.7/site-packages/')

In [3]:
import torch
import torch.nn as nn

In [18]:
lstm1 = nn.LSTM(input_size = 3,
                hidden_size = 5,
                batch_first=True)
x = torch.randn(2, 10, 3)
out, (h_n, c_n) = lstm1(x)

In [19]:
out

tensor([[[-0.1097, -0.1398, -0.0432,  0.0464, -0.0914],
         [ 0.0964, -0.3064, -0.0196,  0.1099, -0.1324],
         [-0.0507, -0.3491,  0.0378,  0.2085, -0.1394],
         [-0.1646, -0.2284, -0.0180,  0.0969, -0.1649],
         [-0.0692, -0.3237, -0.0291,  0.1008, -0.1794],
         [-0.1373, -0.2485,  0.0074,  0.1046, -0.1556],
         [-0.0339, -0.3162, -0.0516,  0.1036, -0.1210],
         [ 0.1223, -0.3908, -0.0467,  0.1253, -0.1527],
         [-0.0770, -0.4228,  0.0262,  0.2330, -0.1392],
         [-0.1225, -0.4484,  0.0432,  0.2269, -0.1339]],

        [[-0.1053,  0.0099,  0.0178,  0.0253, -0.0058],
         [ 0.0951, -0.1609, -0.1057,  0.0101, -0.1136],
         [ 0.2736, -0.0120, -0.0369, -0.0062, -0.0519],
         [-0.0427, -0.0334, -0.1278, -0.1128, -0.1217],
         [-0.1582, -0.1054, -0.1180, -0.0766, -0.0971],
         [-0.0328, -0.2131, -0.1304, -0.0080, -0.1426],
         [-0.1287, -0.3229,  0.0767,  0.2300, -0.0659],
         [-0.1979, -0.3231,  0.1450,  0.2714, 

In [20]:
lstm2 = nn.LSTM(input_size = 5,
                hidden_size = 5,
                batch_first=True)
out2, (h2_n, c2_n) = lstm2(out)

In [21]:
out2

tensor([[[ 0.0602, -0.0151, -0.0334,  0.0179, -0.0550],
         [ 0.0702, -0.0186, -0.0250,  0.0142, -0.0955],
         [ 0.0706, -0.0075, -0.0035,  0.0124, -0.1097],
         [ 0.0809, -0.0202, -0.0189,  0.0148, -0.0903],
         [ 0.0789, -0.0237, -0.0173,  0.0118, -0.0924],
         [ 0.0818, -0.0257, -0.0215,  0.0160, -0.0874],
         [ 0.0801, -0.0249, -0.0188,  0.0112, -0.0954],
         [ 0.0699, -0.0243, -0.0093,  0.0038, -0.1157],
         [ 0.0650, -0.0056,  0.0145,  0.0037, -0.1210],
         [ 0.0635,  0.0081,  0.0300,  0.0029, -0.1228]],

        [[ 0.0678, -0.0181, -0.0478,  0.0252, -0.0526],
         [ 0.0864, -0.0401, -0.0601,  0.0205, -0.0763],
         [ 0.0964, -0.0633, -0.0788,  0.0256, -0.0974],
         [ 0.1052, -0.0844, -0.1052,  0.0211, -0.0554],
         [ 0.1055, -0.0798, -0.1053,  0.0221, -0.0352],
         [ 0.0992, -0.0711, -0.0871,  0.0196, -0.0480],
         [ 0.0840, -0.0239, -0.0305,  0.0226, -0.0818],
         [ 0.0795,  0.0072,  0.0071,  0.0233, 